In [2]:
import numpy as np
import pandas as pd
import os
PATH = 'C:/Workspace/python/Data_Science/dataA/ProDS특강/'
file_list = os.listdir(PATH)
print(file_list)

['Dataset.zip', 'Dataset_01.csv', 'Dataset_02.csv', 'Dataset_03.csv', 'Dataset_04.csv', 'Dataset_05.csv', 'Dataset_05_item_list.csv', 'Dataset_05_Mart_POS.csv', 'Dataset_06.csv', 'Dataset_07.csv', 'Dataset_08.csv', 'Dataset_09.csv', 'Dataset_10.csv', 'Dataset_11.csv', 'Dataset_12.csv', 'Dataset_13_test.csv', 'Dataset_13_train.csv', 'Dataset_14.csv', 'Dataset_15_item_list.csv', 'Dataset_15_Mart_POS.csv']


In [86]:
df_11 = pd.read_csv(PATH + file_list[-7])
# df_11.info()
print(df_11.columns)
q1 = df_11.groupby('Country').apply(len)
print(len(q1[q1 < 3]))
country_list = list(q1[q1 == 3].index)
df_11_ch = df_11[df_11.Country.isin(country_list)].reset_index(drop = True).copy()
q2 = pd.pivot_table(df_11_ch, values = 'Happiness_Score', columns = 'year', index = 'Country')
print(q2.columns)
q2['ratio'] = (q2[2017] - q2[2015]) / 2
print(q2['ratio'].nlargest(3).index)

from scipy.stats import f_oneway # 결과 값만 보여줌
from statsmodels.formula.api import ols # 'Happiness_Score~C(year)'.' '.join형태로 넣어줘야함 / y변수를 앞에 넣어줌.
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd # 사후분석, 다중분석에 사용 / 차이가 난다라는 것이 검증되어여 사용

q3 = ols('Happiness_Score~C(year)', data = df_11_ch).fit()
q3.summary()

anova_lm(q3)
out = pairwise_tukeyhsd(df_11_ch['Happiness_Score'], df_11_ch['year'])
print(out)

Index(['Country', 'Happiness_Rank', 'Happiness_Score', 'year'], dtype='object')
20
Int64Index([2015, 2016, 2017], dtype='int64', name='year')
Index(['Latvia', 'Romania', 'Togo'], dtype='object', name='Country')
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
  2015   2016  -0.0112   0.9 -0.3261 0.3038  False
  2015   2017   -0.001   0.9 -0.3159  0.314  False
  2016   2017   0.0102   0.9 -0.3048 0.3251  False
--------------------------------------------------


In [155]:
df_12 = pd.read_csv(PATH + file_list[-6])
x_list = ['Age', 'Dependent_Count', 'is_Married', 'Read_Book_per_Year']
q1 = df_12[x_list]
print(abs(q1.corr().Read_Book_per_Year.drop(['Read_Book_per_Year'])).idxmax(), round(abs(q1.corr().Read_Book_per_Year.drop(['Read_Book_per_Year'])).max(),4))

from sklearn.linear_model import LinearRegression
q3 = df_12.copy()
x_list = ['Age', 'Dependent_Count', 'is_Married']
q3 = q3.drop(q3[q3.Education_Level  == '고졸'].index).reset_index(drop = True)
q3 = q3.drop(q3[q3.Income_Range  == 'X'].index).reset_index(drop = True)
lm = LinearRegression().fit(q3[x_list], q3.Read_Book_per_Year)
lm.coef_

Age 0.7968


array([ 0.79644021, -0.25026871,  0.04962377])

In [260]:
df_15_pos = pd.read_csv(PATH + file_list[-1])
df_15_item = pd.read_csv(PATH + file_list[-2])

date_list = df_15_pos.Date.unique()
q1 = df_15_pos.copy()
q1.Date.value_counts() # 2015-01-21 : 96
print(f"1번 답 : {q1[q1.Date == '2015-01-21'].itemDescription.value_counts().idxmax()}, {q1[q1.Date == '2015-01-21'].itemDescription.value_counts().max()}")

q2 = pd.merge(df_15_pos, df_15_item, how = 'left', left_on = 'itemDescription', right_on = 'prod_nm').drop(['prod_nm', 'frozen'], axis = 1)
q2['month'] = pd.to_datetime(q2.Date).dt.month
q2['day'] = pd.to_datetime(q2.Date).dt.day_name(locale = 'ko_kr')
q2['weekday'] = q2['day'].isin(['금요일', '토요일']) + 0
q2_ch1 = q2[q2['month'].isin([1, 2, 3])].reset_index(drop = True)
q2_ch1_tab = pd.pivot_table(q2_ch1, index = 'Date', columns = 'weekday', values = 'alcohol', aggfunc = sum)

from scipy.stats import ttest_ind
q2_out = ttest_ind(q2_ch1_tab[1].dropna(), q2_ch1_tab[0].dropna(), equal_var=False)
print(f'2번 답 : {round(q2_out.pvalue, 2)}')

q3 = df_15_pos.copy()
product_list = list(q3.itemDescription.value_counts()[:10].index)
main_product_index= q3['itemDescription'].isin(product_list)[q3['itemDescription'].isin(product_list) == True].index
q3 = q3.drop(main_product_index).reset_index(drop = True)
q3['day'] = pd.to_datetime(q3.Date).dt.day_name(locale = 'ko_kr')
q3_tab = pd.pivot_table(q3, values = 'itemDescription', index = ['Date', 'day'], aggfunc = 'count').reset_index()
q3_tab

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

ols1 = ols('itemDescription~day', data = q3_tab).fit()
q3_out = anova_lm(ols1)

print(f"3번 답 : {q3_out['PR(>F)'][0]}")

1번 답 : soda, 7
2번 답 : 0.02
3번 답 : 0.7360681095250561
